In [10]:
# Opracowane na podstawie https://minpy.readthedocs.io/en/latest/tutorial/rnn_mnist.html
using MLDatasets, Flux
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x1dim = reshape(data.features, 28 * 28, :) # reshape 28×28 pixels into a vector of pixels
    yhot  = Flux.onehotbatch(data.targets, 0:9) # make a 10×60000 OneHotMatrix
    Flux.DataLoader((x1dim, yhot); batchsize, shuffle=true)
end

net = Chain(
    RNN((14 * 14) => 64, tanh),
    Dense(64 => 10, identity),
)

Chain(
  Recur(
    RNNCell(196 => 64, tanh),           # 16_768 parameters
  ),
  Dense(64 => 10),                      # 650 parameters
)         # Total: 6 trainable arrays, 17_418 parameters,
          # plus 1 non-trainable, 64 parameters, summarysize 68.406 KiB.

In [8]:
using Statistics: mean  # standard library
function loss_and_accuracy(model, data)
    (x,y) = only(loader(data; batchsize=length(data)))
    Flux.reset!(model)
    ŷ = model(x[  1:196,:])
    ŷ = model(x[197:392,:])
    ŷ = model(x[393:588,:])
    ŷ = model(x[589:end,:])
    loss = Flux.logitcrossentropy(ŷ, y)  # did not include softmax in the model
    acc = round(100 * mean(Flux.onecold(ŷ) .== Flux.onecold(y)); digits=2)
    (; loss, acc, split=data.split)  # return a NamedTuple
end

@show loss_and_accuracy(net, test_data);  # accuracy about 10%, before training

train_log = []
settings = (;
    eta = 15e-3,
    epochs = 5,
    batchsize = 100,
)

opt_state = Flux.setup(Descent(settings.eta), net); 
display(opt_state)

(layers = ((cell = (σ = (), Wi = Leaf(Descent(0.015), nothing), Wh = Leaf(Descent(0.015), nothing), b = Leaf(Descent(0.015), nothing), state0 = Leaf(Descent(0.015), nothing)), state = ()), (weight = Leaf(Descent(0.015), nothing), bias = Leaf(Descent(0.015), nothing), σ = ())),)

loss_and_accuracy(net, test_data) = (loss = 0.1007138f0, acc = 96.82, split = :test)


In [11]:
using ProgressMeter

for epoch in 1:settings.epochs
    @time for (x,y) in loader(train_data, batchsize=settings.batchsize)
        Flux.reset!(net)
        grads = Flux.gradient(model -> let
                ŷ = model(x[  1:196,:])
                ŷ = model(x[197:392,:])
                ŷ = model(x[393:588,:])
                ŷ = model(x[589:end,:])
                Flux.logitcrossentropy(ŷ, y)
            end, net)
        Flux.update!(opt_state, net, grads[1])
    end
    
    loss, acc, _ = loss_and_accuracy(net, train_data)
    test_loss, test_acc, _ = loss_and_accuracy(net, test_data)
    @info epoch acc test_acc
    nt = (; epoch, loss, acc, test_loss, test_acc) 
    push!(train_log, nt)
end
@show loss_and_accuracy(net, train_data);


  3.737354 seconds (835.96 k allocations: 2.650 GiB, 15.26% gc time, 11.65% compilation time)


┌ Info: 1
│   acc = 89.52
│   test_acc = 90.02
└ @ Main g:\PROGRAMOWANIE\RNN\AWID-2024-RNN.ipynb:18


  2.842264 seconds (541.34 k allocations: 2.633 GiB, 6.10% gc time)


┌ Info: 2
│   acc = 91.95
│   test_acc = 92.0
└ @ Main g:\PROGRAMOWANIE\RNN\AWID-2024-RNN.ipynb:18


  2.910507 seconds (541.34 k allocations: 2.633 GiB, 6.13% gc time)


...

  2.824527 seconds (541.34 k allocations: 2.633 GiB, 5.94% gc time)


┌ Info: 4
│   acc = 93.74
│   test_acc = 93.78
└ @ Main g:\PROGRAMOWANIE\RNN\AWID-2024-RNN.ipynb:18


  3.137467 seconds (541.34 k allocations: 2.633 GiB, 16.11% gc time)


┌ Info: 5
│   acc = 94.47
│   test_acc = 94.33
└ @ Main g:\PROGRAMOWANIE\RNN\AWID-2024-RNN.ipynb:18


loss_and_accuracy(net, train_data) = (loss = 0.1955753f0, acc = 94.47, split = :train)
